In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/insurance/insurance.csv")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
dms = pd.get_dummies(df[["sex","smoker","region"]])
dff = df.drop(["sex","smoker","region","charges"],axis=1)
dms

In [ ]:
Ddf = pd.concat([dff,dms],axis=1)
Ddf.head()

In [ ]:
X = Ddf
y = df["charges"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

# Ridge Regression

In [ ]:
RidgeModel = Ridge().fit(X_train,y_train)

In [ ]:
RidgeModel.coef_

In [ ]:
RidgeModel.intercept_

In [ ]:
y_pred = RidgeModel.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test,y_pred)
mse

In [ ]:
r2_score(y_test,y_pred)

# Model Tuning

In [ ]:
lambdas = 10**np.linspace(10,-2,100)*0.5
RidgeCV = RidgeCV(alphas = lambdas, cv=10,scoring="neg_mean_squared_error",normalize=True)  # 10k fold cross validation
RidgeCV.fit(X_train,y_train)

In [ ]:
RidgeCV.alpha_   #best Lambda value for this model

In [ ]:
TunedRidgeModel = Ridge(alpha = RidgeCV.alpha_).fit(X_train,y_train)

In [ ]:
tunedY_pred = TunedRidgeModel.predict(X_test)

In [ ]:
ridgemse = mean_squared_error(y_test,tunedY_pred)
ridgemse

In [ ]:
ridger2 = r2_score(y_test,tunedY_pred)
ridger2

# Lasso Regression

In [ ]:
LassoModel = Lasso().fit(X_train,y_train)

In [ ]:
LassoModel.intercept_

In [ ]:
LassoModel.coef_

In [ ]:
lassoYpred_ = LassoModel.predict(X_test)
lassoYpred_[:5]

In [ ]:
lassomse = mean_squared_error(y_test,lassoYpred_)
lassomse

In [ ]:
lassor2 = r2_score(y_test,lassoYpred_)
lassor2

# Model Tuning

In [ ]:
LassoCV = LassoCV(cv=10,max_iter = 1000000)  #10k fold validation
LassoCV.fit(X_train,y_train)

In [ ]:
LassoCV.alpha_ #best lambda

In [ ]:
tunedlasso = Lasso(alpha=LassoCV.alpha_).fit(X_train,y_train)

In [ ]:
tunedlassoYpred_ = tunedlasso.predict(X_test)
tunedlassoYpred_[:5]


In [ ]:
lassomse = mean_squared_error(y_test,tunedlassoYpred_)
lassomse

In [ ]:
lassor2 = r2_score(y_test,tunedlassoYpred_)
lassor2

# ElasticNet Regression

In [ ]:
enet_model = ElasticNet().fit(X_train,y_train)

In [ ]:
enetYpred_ = enet_model.predict(X_test)

In [ ]:
mean_squared_error(y_test,enetYpred_)

In [ ]:
r2_score(y_test,enetYpred_)

# Model Tuning

In [ ]:
EnetCV = ElasticNetCV(cv=10).fit(X_train,y_train)

In [ ]:
EnetCV.alpha_ #best alpha

In [ ]:
TunedEnet = ElasticNet(alpha=EnetCV.alpha_).fit(X_train,y_train)

In [ ]:
tunedenetYpred_ = TunedEnet.predict(X_test)

In [ ]:
enetmse = mean_squared_error(y_test,tunedenetYpred_)
enetmse

In [ ]:
enetr2 = r2_score(y_test,tunedenetYpred_)
enetr2

# Final

In [ ]:
print(f"Ridge Mean Squared Error: {ridgemse}\nRidge R2 Score: {ridger2}\nLasso Mean Squared Error: {lassomse}\nLasso R2 Score: {lassor2}\nElasticNet Mean Squared Error: {enetmse}\nElasticNet R2 Score: {enetr2}\n")